## 任务转交

实现方式：
1. 主 agent 设置 handoffs 参数，设定接受移交任务的 agent。参考c2.ipynb
2. 通过 handoff() 实现移交

In [ ]:
from agents import Agent, Runner, handoff, RunContextWrapper, set_tracing_disabled
import asyncio
from agents.extensions.models.litellm_model import LitellmModel
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


def on_handoff(ctx: RunContextWrapper[None]):
    print("Handoff called")


history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)


handoff_obj = handoff(
    agent=math_tutor_agent,
    on_handoff=on_handoff,
    tool_name_override="custom_handoff_tool",
    tool_description_override="Custom description",
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, handoff_obj],
    model=llm,
)

async def main():
    result = await Runner.run(triage_agent, "What is the capital of France?")
    print(result.final_output)

await main()

## 追踪

Agent 在对话或任务过程中会涉及多轮交互、状态变化、外部工具调用等。如果不持续追踪，Agent 可能“跑偏”、忘记上下文，导致行为失误或效率降低。因此，需要设计机制持续 记录、回顾、更新 当前“状态”与“历史”。

记录智能体运行期间的所有事件：包括大模型生成内容、工具调用、交接流程、防护机制触发以及自定义事件等

### 禁用追踪

1. 通过设置环境变量设置全局关闭追踪功能 `OPENAI_AGENTS_DISABLE_TRACING=1`
2. 针对单个程序文件禁用追踪 `set_tracing_disabled(disabled=False)`
3. 针对单次运行禁用追踪：将 agents.run.RunConfig.tracing_disabled 设为 True

### 追踪与跨度

Trace（追踪）包含多个span(跨度)，记录一次完整任务或流程，span是更细粒度的时间段，比如一次llm调用操作

### 追踪的代码实现

#### 将多次 run() 调用归入同一条规则记录

在 logging 中，也可以设置让多个 python 文件的运行日志记录到一个文件中

In [ ]:
from agents import Agent, Runner, trace

async def main():
    agent = Agent(name="Joke generator", instructions="Tell funny jokes.")

    with trace("Joke workflow"): 
        first_result = await Runner.run(agent, "Tell me a joke")
        second_result = await Runner.run(agent, f"Rate this joke: {first_result.final_output}")
        print(f"Joke: {first_result.final_output}")
        print(f"Rating: {second_result.final_output}")

## 护栏

为 agent 设置处理任务的边界，具体：检查和验证用户输入；防止恶意使用；节省成本和时间

### 类型

| 维度         | 输入护栏                                 | 输出护栏                           |
| ------------ | ---------------------------------------- | ---------------------------------- |
| **装饰器**   | `@input_guardrail`                       | `@output_guardrail`                |
| **函数参数** | `input: str \| list[TResponseInputItem]` | `output: MessageOutput`            |
| **数据源**   | 用户原始输入                             | 智能体处理后的输出                 |
| **异常类型** | `InputGuardrailTripwireTriggered`        | `OutputGuardrailTripwireTriggered` |
| **配置属性** | `input_guardrails=[...]`                 | `output_guardrails=[...]`          |

In [ ]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    InputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    set_tracing_disabled,
    TResponseInputItem,
    input_guardrail,
)

import os
from dotenv import load_dotenv
import asyncio
from agents.extensions.models.litellm_model import LitellmModel

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

class MathHomeworkOutput(BaseModel):
    is_math_homework: bool
    reasoning: str

guardrail_agent = Agent( 
    name="Guardrail check",
    instructions="Check if the user is asking you to do their math homework.",
    model=llm,
    output_type=MathHomeworkOutput,
)


@input_guardrail
async def math_guardrail(ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, input, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output, 
        tripwire_triggered=result.final_output.is_math_homework,
    )


agent = Agent(  
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    model=llm,
    input_guardrails=[math_guardrail],
)

async def main():
    # This should trip the guardrail
    try:
        await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
        print("Guardrail didn't trip - this is unexpected")

    except InputGuardrailTripwireTriggered:
        print("Math homework guardrail tripped")

await main()

In [ ]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    set_tracing_disabled,
    Runner,
    output_guardrail,
)

import os
from dotenv import load_dotenv
import asyncio
from agents.extensions.models.litellm_model import LitellmModel

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

class MessageOutput(BaseModel): 
    response: str

class MathOutput(BaseModel): 
    reasoning: str
    is_math: bool

guardrail_agent = Agent(
    name="Guardrail check",
    instructions="Check if the output includes any math.",
    model=llm,
    output_type=MathOutput,
)

@output_guardrail
async def math_guardrail(ctx: RunContextWrapper, agent: Agent, output: MessageOutput) -> GuardrailFunctionOutput:
    result = await Runner.run(guardrail_agent, output.response, context=ctx.context)

    return GuardrailFunctionOutput(
        output_info=result.final_output,
        tripwire_triggered=result.final_output.is_math,
    )

agent = Agent( 
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    model=llm,
    output_guardrails=[math_guardrail],
    output_type=MessageOutput,
)

async def main():
    # This should trip the guardrail
    try:
        await Runner.run(agent, "Hello, can you help me solve for x: 2x + 3 = 11?")
        print("Guardrail didn't trip - this is unexpected")

    except OutputGuardrailTripwireTriggered:
        print("Math output guardrail tripped")

await main()

## 编排

编排（Orchestration）：程序中 agent 的运行流畅，哪些智能体执行任务、以何种顺序执行、后续操作是什么

两种方式：

1. 大模型自主决策。
2. 代码编排。在执行速度、成本控制和性能表现方面提供更确定性的结果